## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тестовое множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

In [1]:
%cd C:\Users\Michel\DataspellProjects\dsProject\docs_classif\task6

C:\Users\Michel\DataspellProjects\dsProject\docs_classif\task6


# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации документа по категориям, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

Task1

In [2]:
import pymorphy2
import pandas as pd
from tqdm import tqdm
import numpy as np
import re
import os
import nltk
import string

In [ ]:
morph = pymorphy2.MorphAnalyzer()
nltk.download('stopwords')
stop_words=nltk.corpus.stopwords.words('russian')
stop_words

Сделаем датасет, предварительно убрав стоп слова и пунктуацию. Приведем слова к нормальной форме с помощью pymorphy2

In [5]:
df_train = pd.DataFrame({'theme':[],'text':[],'text_normalized':[]})
idx = 0
vocalulary_train = []
with open('news_train.txt', encoding='utf8') as f_train:
  for line in f_train:
    line = re.sub(r'[^\w\s]','',line)
    line = re.split(r'\t', line)
    text = ' '.join(line[1:])
    normal_line =''
    for word in text.split():
      if word in stop_words:
        continue
      normal_word = morph.parse(word)[0].normal_form
      vocalulary_train.append(normal_word)
      normal_line=' '.join((normal_line,normal_word))
    df_train.loc[idx] = [line[0],text,normal_line]
    idx+=1

In [6]:
df_test = pd.DataFrame({'theme':[],'text':[],'text_normalized':[]})
idx = 0
vocalulary_test = []
with open('news_test.txt', encoding='utf8') as f_test:
  for line in f_test:
    line = re.sub(r'[^\w\s]','',line)
    line = re.split(r'\t', line)
    text = ' '.join(line[1:])
    normal_line =''
    for word in text.split():
      if word in stop_words:
        continue
      normal_word = morph.parse(word)[0].normal_form
      vocalulary_test.append(normal_word)
      normal_line=' '.join((normal_line,normal_word))
    df_test.loc[idx] = [line[0],text,normal_line]
    idx+=1

In [7]:
df_train.head()

,theme,text,text_normalized
0,sport,Овечкин пожертвовал детской хоккейной школе ав...,овечкин пожертвовать детский хоккейный школа ...
1,culture,Рекордно дорогую статую майя признали подделко...,рекордно дорогой статуя майя признать подделк...
2,science,Samsung представила флагман в защищенном корпу...,samsung представить флагман защитить корпус ю...
3,sport,С футболиста Спартака сняли четырехматчевую ди...,с футболист спартак снять четырехматчевой дис...
4,media,Hopes Fears объединится с The Village Интерне...,hopes fears объединиться the village интернет...


In [8]:
df_test.head()

,theme,text,text_normalized
0,culture,Жительница Ямала победила в первом песенном ко...,жительница ямал победить первый песенный конк...
1,media,Почти половина Twitterпользователей никогда не...,почти половина twitterпользователь писать соо...
2,media,Билайн начал рекламу роуминга под песенку Трол...,билайн начать реклама роуминг песенка трололо...
3,business,Saipem потеряла 12 миллиарда евро изза отмены ...,saipem потерять 12 миллиард евро изз отмена ю...
4,culture,Вин Дизель назвал Форсаж 7 достойным Оскара Ак...,вино дизель назвать форсаж 7 достойный оскар ...


In [9]:
vocabulary_train = set(vocalulary_train)
vocabulary_test = set(vocalulary_test)

Task2

In [10]:
text_train = '  '.join(df_train['text_normalized'])
text_test = '  '.join(df_test['text_normalized'])

In [11]:
# with open('normalized_train.txt','w', encoding='utf8') as f_train:
#   f_train.write(text_train)

In [12]:
with open('normalized_test.txt','w', encoding='utf8') as f_test:
  f_test.write(text_test)

Получим эмбеддинги слов с помощью fasttext

In [13]:
import fasttext
model = fasttext.train_unsupervised('normalized_train.txt', model='cbow')

In [14]:
print(len(model.words))

27925


Выведем 10 ближайших слов к словам 'кошка' и 'собака', тем самым продемострируем семантические ассоциации 

In [15]:
model.get_nearest_neighbors('кошка')

[(0.9339474439620972, 'галушка'),
 (0.9324925541877747, 'мушка'),
 (0.9273046255111694, 'ловушка'),
 (0.9190776348114014, 'кукушка'),
 (0.9167001843452454, 'лягушка'),
 (0.9102931022644043, 'бабушка'),
 (0.9070371389389038, 'шкатулка'),
 (0.899564802646637, 'собака'),
 (0.8991677761077881, 'кормушка'),
 (0.8927677869796753, 'чашка')]

In [16]:
model.get_nearest_neighbors('собака')

[(0.8995648622512817, 'кошка'),
 (0.8853743076324463, 'котёнок'),
 (0.8842496275901794, 'козлёнок'),
 (0.876847505569458, 'лосёнок'),
 (0.8726280331611633, 'собачка'),
 (0.8709497451782227, 'кличка'),
 (0.8495503067970276, 'тигрёнок'),
 (0.8482656478881836, 'львёнок'),
 (0.8463788032531738, 'ягнёнок'),
 (0.8460367321968079, 'кот')]

Task 3

Обучим SVM и лог регрессию. Получим эмбеддинг предложения за счет усреднения эмбеддингов слов в предложении

In [17]:
list_theme = list(set(df_train['theme']))
theme_idx = dict()
for i in range(len(list_theme)):
  theme_idx[list_theme[i]]=i
list_theme_train = []
for theme in df_train['theme']:
  list_theme_train.append(theme_idx[theme])
list_theme_test = []
for theme in df_test['theme']:
  list_theme_test.append(theme_idx[theme])

In [18]:
df_list = []
targets = []
for line in df_train['text_normalized']:
  seq_emb = np.zeros((1,100))
  count=0
  for word in line.split():
    seq_emb+=np.array(model[word])
    count+=1
  seq_emb=seq_emb/count
  df_list.append(seq_emb)
len(df_list)

15000

In [19]:
df_data_train=pd.DataFrame(data=np.squeeze(np.array(df_list)))
df_data_train.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.846750,-0.211703,-0.343574,-0.318169,-0.313551,0.409213,-0.077015,-0.783134,-0.372726,-0.102156,...,0.328988,-0.135662,-0.149695,-0.123620,0.186095,0.118507,0.303843,0.080714,0.160952,-0.156663
1,-0.406527,-0.086954,0.108069,-0.218579,0.312164,0.449005,0.192080,-0.067015,-0.272677,0.317649,...,-0.379923,0.064984,0.250864,0.047013,0.235075,-0.583004,0.033285,0.477899,0.325165,-0.202244
2,-0.128463,0.081608,-0.509347,-0.072902,0.422272,0.075654,-0.072248,0.200054,-0.548765,0.057418,...,-0.578394,0.372366,-0.138986,0.549562,0.319726,-0.272478,-0.323735,-0.271193,-0.066781,-0.230923
3,-1.397803,0.104870,-0.291781,-0.399679,0.207858,0.393510,0.441083,-0.622452,-0.094257,-0.501257,...,0.071181,0.035838,-0.558536,-0.480131,0.387728,0.218733,0.046231,-0.189890,0.175672,-0.237542
4,-0.319571,-0.744991,-0.573416,-0.018779,0.608332,0.609178,0.135325,-0.035312,-0.738355,0.354851,...,0.316036,0.339726,0.269105,0.469213,0.216651,0.085785,0.021640,0.331390,0.439917,-0.481991


In [20]:
df_data_train.shape

(15000, 100)

In [21]:
df_list = []
targets = []
for line in df_test['text_normalized']:
  seq_emb = np.zeros((1,100))
  count=0
  for word in line.split():
    seq_emb+=np.array(model[word])
    count+=1
  seq_emb=seq_emb/count
  df_list.append(seq_emb)
df_data_test=pd.DataFrame(data=np.squeeze(np.array(df_list)))
df_data_test.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.717223,-0.242777,-0.168939,-0.166373,-0.071919,0.405911,0.416954,-0.196197,-0.027091,-0.234620,...,-0.048485,0.021613,0.294594,-0.193145,0.364677,-0.534842,0.304378,0.396094,0.120990,0.070339
1,-0.813868,0.080107,-0.883001,-0.427814,0.908722,0.270789,0.321877,-0.107676,-0.816036,0.419013,...,-0.257851,0.000144,0.442114,0.012822,-0.223277,-0.422229,-0.179858,0.026250,0.381324,-0.727991
2,-0.686143,-0.367384,-0.176150,-0.234342,0.263433,0.394117,0.502982,0.010007,-0.630374,-0.026753,...,0.058678,0.233279,0.017320,-0.065201,0.292025,-0.241506,0.226455,0.297647,0.039798,-0.504876
3,-1.271873,0.443849,-0.104368,-0.031319,0.767693,0.548564,0.434810,0.347156,-0.010105,-0.099858,...,-0.600785,0.176777,0.322054,0.124736,0.388106,-0.410580,-0.431753,0.422078,0.227545,-0.420445
4,-0.517246,-0.374558,0.060699,0.082716,0.282908,0.588812,0.429589,0.031003,-0.246634,-0.291885,...,-0.112912,0.171904,-0.501609,-0.075209,0.503934,-0.459679,0.123241,0.331275,0.075802,-0.206667


SVM

In [22]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto',probability=True))
clf.fit(df_data_train,list_theme_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto', probability=True))])

Метрики:

In [23]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
pred = clf.predict(df_data_test)
pred_proba = clf.predict_proba(df_data_test)
print('conf_matrix:',confusion_matrix(list_theme_test,pred))


conf_matrix: [[379   0   5   0   0   3   3   0  13  23]
 [  1  43   2   2   0   0   2   1   0   1]
 [  6   0 382   3   0  48   5   0  14   8]
 [  1   2   4  26   0   1  51   1   4   0]
 [  0   0   0   0 411   4   0   0   0   8]
 [  0   0   6   1   0 223   4   1   6   4]
 [  1   0   2  11   2   5 381   3  17   4]
 [  2   0   2   0   0   0   1  38   3   8]
 [  5   1  11   2   5   4  16   1 338  20]
 [ 22   3   8   0   2   3   2   1  13 361]]


In [24]:
print('accuracy:', accuracy_score(pred,list_theme_test))
print('roc_auc:',roc_auc_score(list_theme_test,pred_proba, multi_class='ovr'))

accuracy: 0.8606666666666667
roc_auc: 0.9855936313877265


Логистическая регрессия 

In [25]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(df_data_train,list_theme_train)

c:\users\michel\appdata\roaming\jetbrains\dataspell2021.3\projects\workspace\venv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

Метрики

In [26]:
pred = clf.predict(df_data_test)
print('conf_matrix:',confusion_matrix(list_theme_test,pred))


conf_matrix: [[375   0   4   0   0   5   1   1  14  26]
 [  3  41   3   2   0   0   0   1   1   1]
 [  7   0 396   1   1  32   4   0  11  14]
 [  1   0   4  31   0   3  46   0   5   0]
 [  0   0   0   0 410   3   0   0   1   9]
 [  1   0  21   1   1 204   5   2   6   4]
 [  0   1   4  21   2   5 370   2  18   3]
 [  2   0   4   1   0   1   3  32   2   9]
 [  6   1  18   1   4   4  14   1 333  21]
 [ 21   3  11   0   2   6   0   3  19 350]]


In [28]:
pred_proba=clf.predict_proba(df_data_test)

In [29]:
print('accuracy:', accuracy_score(pred,list_theme_test))
print('roc_auc:',roc_auc_score(list_theme_test,pred_proba, multi_class='ovr'))

accuracy: 0.8473333333333334
roc_auc: 0.9832361503439732


Как видим, мы получили довольно хорошие скоры на тестовой выборке. SVM показал себя чуть лучше, чем лог регрессия